# Movie Analytics Orchestrator

This notebook orchestrates the movie analytics workflow, allowing you to:
1. Train the model
2. Predict success metrics for given budget and genres
3. Predict ideal budget and genres for desired metrics

In [ ]:
import os
import json
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from IPython.display import clear_output

In [ ]:
budget = 100000000
genres = ['Action', 'Adventure']

desired_revenue = 30000000
desired_popularity = 40

In [12]:
input = {
    'budget': budget,
    'genres': genres,
    'desired_revenue': desired_revenue,
    'desired_popularity': desired_popularity
}
with open('input.json', 'w') as f:
    json.dump(input, f)

In [ ]:
class MovieAnalytics:
    def __init__(self):
        self.models_dir = 'models'
        if not os.path.exists(self.models_dir):
            os.makedirs(self.models_dir)
    
    def execute_notebook(self, notebook_path, parameters=None):
        with open(notebook_path) as f:
            nb = nbformat.read(f, as_version=4)
        
        if parameters:
            parameter_cell = {
                'cell_type': 'code',
                'metadata': {},
                'source': '\n'.join([f'{k} = {v}' for k, v in parameters.items()])
            }
            nb.cells.insert(1, nbformat.from_dict(parameter_cell))
        
        ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
        ep.preprocess(nb, {'metadata': {'path': './'}})
        
        clear_output()
        return nb
    
    def train_model(self):
        print("Training model...")
        self.execute_notebook('2_model-training.ipynb')
        print("Model training completed successfully!")
        return
    
    def predict_success(self, budget, genres):
        print(f"Predicting success for ${budget:,} budget with genres {genres}...")
        parameters = {
            'budget': budget,
            'desired_genres': genres
        }
        self.execute_notebook('3_predict-success.ipynb', parameters)
        print("Prediction completed successfully!")
        with open('results/predicted_success_output.json', 'r') as f:
            predictions = json.load(f)
        print('Predicted revenue: $%.2f' % predictions['expected_revenue'])
        print('Predicted popularity: %.2f' % predictions['expected_popularity'])
        return
    
    def predict_parameters(self, desired_revenue, desired_popularity):
        print(f"Predicting budget for ${desired_revenue:,} revenue and {desired_popularity} popularity...")
        parameters = {
            'desired_revenue': desired_revenue,
            'desired_popularity': desired_popularity
        }
        self.execute_notebook('4_predict-parameters.ipynb', parameters)
        print("Prediction completed successfully!")
        with open('results/predicted_parameters_output.json', 'r') as f:
            predictions = json.load(f)
        print('Estimated budget: $%.2f' % predictions['ideal_budget'])
        print('Suggested genres:', predictions['suggested_genres'])
        return 

In [14]:
# Initialize the MovieAnalytics class
analytics = MovieAnalytics()

## 1. Train the Model
Run this cell to train the model if you haven't already or want to retrain it.

In [15]:
analytics.train_model()

Model training completed successfully!


## 2. Predict Success Metrics
Predict expected revenue and popularity for a given budget and genres.

In [16]:
analytics.predict_success(budget, genres)

Prediction completed successfully!
Predicted revenue: $297526464.36
Predicted popularity: 40.13


## 3. Predict Ideal Parameters
Predict the ideal budget and suggest genres for desired revenue and popularity metrics.

In [17]:
analytics.predict_parameters(desired_revenue, desired_popularity)

Prediction completed successfully!
Estimated budget: $11652576.60
Suggested genres: ['Action', 'Science Fiction', 'Adventure']
